In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from fastDamerauLevenshtein import damerauLevenshtein

### read data

In [2]:
data = open('data.txt', 'rb').read().decode(encoding='utf-8')

In [3]:
data

'pocgkr\ncgwpcfks\nkfsfkspwg\npfkswkfsococg\nwptgkfsoc\npwfkg\npowcer\ncoprerk\nwopofkkskf\nopwpwgcoc\npocrfksfks\ncgpogpoc\npwocgfkfsrere\nocwgrk\nwpgsfk\npwofsg\npocpocwgkfs\npocgwg\npwgockfsfksret\nocpwgfkskfsrere\ncopwgwpresk\npwterfkf\npocwggpfkfksrer\nwpofcgkrpwgfksocr\ncwpfksor\npwfksorc\npocgwkfkfser\npowctgcog\npowgcpo\npwfkt\npwokfkrg\ncopcogpgwfksr\npgsfkskfg\nwgptocopwg\npowcgre\nperkscwgtf\npcgworkfkfs\npopocgkr\npwtgcfkfksr\npocwkf\npgkfkf\ncwopg\npowcgkfkfr\nwpgfksk\npocwcpoere\npwkf\ncptgo\npocfkswgr\npowcfkfk\npwgfksfk\ngpwfks\ncgwpw\ncwopgtrekfksksf\npocsfkw\nocpwgfkf\npwgterkfsoc\npocgwrkfs\ntgcwpro\npocgkfskfr\npgcow\nopwg\nwpgcowpcfksre\nre\nptwgfks\npocrfkfk\npockr\npockfskfswgre\ntgpcowpgpwcog\ngwpfrkst\npfkfksg\npwogsfkg\npwkf\ngwpocoskfsfk\nwtpcockfkf\nwpgfkfkre\npococwtrfkfkg\npwocfksfksr\npwgerks\npoctfkfkrwg\npwfksocr\ntnpzc\ntnpcz\ntnpzc\ntnzpc\npmz\nxmbzd\npImz\nmzxp\npbz\npgz\npnzga\ndnzgdp\npndz\nnzpdgds\nmcddxp\npcfkdddaag\ndagpk\npc\ngkfadp\npfgds\nbcs

### generate list of sequences

In [4]:
data_split = data.splitlines()

### remove newline chars

In [5]:
data2 = data.replace('\n','')
data2

'pocgkrcgwpcfkskfsfkspwgpfkswkfsococgwptgkfsocpwfkgpowcercoprerkwopofkkskfopwpwgcocpocrfksfkscgpogpocpwocgfkfsrereocwgrkwpgsfkpwofsgpocpocwgkfspocgwgpwgockfsfksretocpwgfkskfsrerecopwgwpreskpwterfkfpocwggpfkfksrerwpofcgkrpwgfksocrcwpfksorpwfksorcpocgwkfkfserpowctgcogpowgcpopwfktpwokfkrgcopcogpgwfksrpgsfkskfgwgptocopwgpowcgreperkscwgtfpcgworkfkfspopocgkrpwtgcfkfksrpocwkfpgkfkfcwopgpowcgkfkfrwpgfkskpocwcpoerepwkfcptgopocfkswgrpowcfkfkpwgfksfkgpwfkscgwpwcwopgtrekfksksfpocsfkwocpwgfkfpwgterkfsocpocgwrkfstgcwpropocgkfskfrpgcowopwgwpgcowpcfksrereptwgfkspocrfkfkpockrpockfskfswgretgpcowpgpwcoggwpfrkstpfkfksgpwogsfkgpwkfgwpocoskfsfkwtpcockfkfwpgfkfkrepococwtrfkfkgpwocfksfksrpwgerkspoctfkfkrwgpwfksocrtnpzctnpcztnpzctnzpcpmzxmbzdpImzmzxppbzpgzpnzgadnzgdppndznzpdgdsmcddxppcfkdddaagdagpkpcgkfadppfgdsbcsfgkappbsfkcgapbgafkcsgpbcksfapbfkscgapbsgkafcpbcskfgapsfkagcbpfkscagbpfskagcbpskfcagbpfkscagbpsfkgacbpfkbscgapcgafkbsgpabkfscpabkgcfsgcpabkfspasbkfgcpabcgksfsfkpcgabspgbkfcaspakgfcbcgbkpasffkspcbagpkf

In [6]:
vocabulary = sorted(set(data2))
#vocabulary.remove('\n')
len(vocabulary)

20

### Vectorize text

In [7]:
example_texts = ['abcdefg', 'xhz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'h', b'z']]>

In [8]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocabulary))

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[3, 4, 5, 6, 7, 8, 9], [20, 10, 21]]>

In [10]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'h', b'z']]>

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xhz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Create training samples + targets

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(data2, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1412,), dtype=int64, numpy=array([15, 14,  5, ...,  6,  9,  4])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

p
o
c
g
k
r
c
g
w
p


In [16]:
seq_length = 8
examples_per_epoch = len(data)//(seq_length+1)

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor([b'p' b'o' b'c' b'g' b'k' b'r' b'c' b'g' b'w'], shape=(9,), dtype=string)


In [18]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'pocgkrcgw'
b'pcfkskfsf'
b'kspwgpfks'
b'wkfsococg'
b'wptgkfsoc'


In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'pocgkrcg'
Target: b'ocgkrcgw'


### Create training batches

In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 8), (64, 8)), types: (tf.int64, tf.int64)>

### Build model

In [23]:
# Length of the vocabulary in chars
vocab_size = len(vocabulary)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else: 
            return x

In [25]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### Try out model

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 8, 22) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  5632      
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  22550     
Total params: 3,966,486
Trainable params: 3,966,486
Non-trainable params: 0
_________________________________________________________________


In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [29]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'fbakpfkc'

Next Char Predictions:
 b'csIowaod'


### Train model

In [30]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 8, 22)  # (batch_size, sequence_length, vocab_size)
Mean loss:         3.090815


In [32]:
tf.exp(mean_loss).numpy()

21.994999

In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
epochs = 300

In [35]:
history = model.fit(dataset, epochs=epochs)

Epoch 1/300
2/2 [==============================] - 0s 60ms/step - loss: 3.0731
Epoch 2/300
2/2 [==============================] - 0s 61ms/step - loss: 2.9765
Epoch 3/300
2/2 [==============================] - 0s 56ms/step - loss: 2.7828
Epoch 4/300
2/2 [==============================] - 0s 56ms/step - loss: 2.7024
Epoch 5/300
2/2 [==============================] - 0s 60ms/step - loss: 2.6310
Epoch 6/300
2/2 [==============================] - 0s 61ms/step - loss: 2.5805
Epoch 7/300
2/2 [==============================] - 0s 59ms/step - loss: 2.5478
Epoch 8/300
2/2 [==============================] - 0s 59ms/step - loss: 2.4671
Epoch 9/300
2/2 [==============================] - 0s 57ms/step - loss: 2.3919
Epoch 10/300
2/2 [==============================] - 0s 62ms/step - loss: 2.3450
Epoch 11/300
2/2 [==============================] - 0s 56ms/step - loss: 2.3157
Epoch 12/300
2/2 [==============================] - 0s 57ms/step - loss: 2.2143
Epoch 13/300
2/2 [==============================]

2/2 [==============================] - 0s 58ms/step - loss: 0.4038
Epoch 104/300
2/2 [==============================] - 0s 60ms/step - loss: 0.3909
Epoch 105/300
2/2 [==============================] - 0s 55ms/step - loss: 0.3941
Epoch 106/300
2/2 [==============================] - 0s 57ms/step - loss: 0.3925
Epoch 107/300
2/2 [==============================] - 0s 59ms/step - loss: 0.3948
Epoch 108/300
2/2 [==============================] - 0s 53ms/step - loss: 0.3920
Epoch 109/300
2/2 [==============================] - 0s 54ms/step - loss: 0.3874
Epoch 110/300
2/2 [==============================] - 0s 56ms/step - loss: 0.3808
Epoch 111/300
2/2 [==============================] - 0s 57ms/step - loss: 0.3808
Epoch 112/300
2/2 [==============================] - 0s 55ms/step - loss: 0.3752
Epoch 113/300
2/2 [==============================] - 0s 62ms/step - loss: 0.3831
Epoch 114/300
2/2 [==============================] - 0s 53ms/step - loss: 0.3763
Epoch 115/300
2/2 [=======================

2/2 [==============================] - 0s 54ms/step - loss: 0.3245
Epoch 205/300
2/2 [==============================] - 0s 52ms/step - loss: 0.3235
Epoch 206/300
2/2 [==============================] - 0s 55ms/step - loss: 0.3233
Epoch 207/300
2/2 [==============================] - 0s 55ms/step - loss: 0.3243
Epoch 208/300
2/2 [==============================] - 0s 53ms/step - loss: 0.3283
Epoch 209/300
2/2 [==============================] - 0s 56ms/step - loss: 0.3192
Epoch 210/300
2/2 [==============================] - 0s 53ms/step - loss: 0.3159
Epoch 211/300
2/2 [==============================] - 0s 57ms/step - loss: 0.3222
Epoch 212/300
2/2 [==============================] - 0s 58ms/step - loss: 0.3172
Epoch 213/300
2/2 [==============================] - 0s 52ms/step - loss: 0.3235
Epoch 214/300
2/2 [==============================] - 0s 56ms/step - loss: 0.3244
Epoch 215/300
2/2 [==============================] - 0s 55ms/step - loss: 0.3237
Epoch 216/300
2/2 [=======================

### Generate prediction

In [36]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature=temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices = skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())]) 
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits] 
        predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [37]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [38]:
states = None
next_char = tf.constant(['p'])
result = [next_char]

for n in range(8):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)

print(result[0].numpy().decode('utf-8'))

pococwtrf


### Prequential method

In [39]:
# test generation of predictions in loop for all first chars in list

results = [[] for line in data_split]

for line in range(0, len(data_split)):
    states = None
    next_char = tf.constant([data_split[line][0]])
    results[line] = [next_char]
    
    for n in range(0, len(data_split[line]) - 1):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        results[line].append(next_char)
    results[line] = tf.strings.join(results[line])
    results[line] = results[line][0].numpy().decode('utf-8')

In [40]:
# check if predicted length = observed length
[len(line) for line in results] == [len(line) for line in data_split]

True

In [41]:
# generate prediction for next char, compare to observed, determine error - until length of observed is reached

results = [[] for line in data_split]
errors = [[] for x in data_split]

for line in range(0, len(data_split)):
    i = 1
    states = None
    
    while i < len(data_split[line]):
        next_char = tf.constant([data_split[line][:i]])
        results[line] = [next_char]
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        results[line].append(next_char)
        predicted = tf.strings.join(results[line])
        predicted = predicted[0].numpy().decode('utf-8')
        observed = data_split[line][:i+1]
        error = 1 - damerauLevenshtein(predicted, observed)
        #print(predicted, observed, error)
        
        errors[line].append(error)
        i +=1
        
    results[line] = tf.strings.join(results[line])
    results[line] = results[line][0].numpy().decode('utf-8')

In [42]:
error_list = [np.median(error) for error in errors]

In [45]:
summed_error = [sum(error) for error in errors]
np.mean(summed_error)

1.1753496834501735

In [43]:
np.mean([np.median(error) for error in errors])

0.15319473736140402

In [44]:
np.median([np.mean(error) for error in errors])

0.1773809523809524

In [46]:
with open('results/rnn_prequential.txt', 'w') as f:
    f.write(str(error_list))